# Power Plant Python Load
### Loads EIA (Energy Information Agency) Power Plant Generation data from two datasets and joins on their EIA Plant ID, cleans and organizes, then plots in Plot.ly
<br>
### To Do: Group final Combined Data Set

In [1]:
# IMPORT CSV and Pandas
import csv
import pandas as pd
import numpy as np
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import json
import time

In [2]:
#state conversion json
stateJSON ='{"1": "AL","2": "AK","4": "AZ","5": "AR","6": "CA","8": "CO","9": "CT","10": "DE","11": "DC","12": "FL","13": "GA","15": "HI","16": "ID","17": "IL","18": "IN","19": "IA","20": "KS","21": "KY","22": "LA","23": "ME","24": "MD","25": "MA","26": "MI","27": "MN","28": "MS","29": "MO","30": "MT","31": "NE","32": "NV","33": "NH","34": "NJ","35": "NM","36": "NY","37": "NC","38": "ND","39": "OH","40": "OK","41": "OR","42": "PA","44": "RI","45": "SC","46": "SD","47": "TN","48": "TX","49": "UT","50": "VT","51": "VA","53": "WA","54": "WV","55": "WI","56": "WY"}'
stateJSON = json.loads(stateJSON)

#Update 3/30: changing to DF
stateDF = pd.DataFrame.from_dict(stateJSON, orient='index')
stateDF.rename(columns={0:'State'}, inplace=True)
stateDF['StateID'] = stateDF.index

reader=[]
start_year=1970
end_year=2018

total_start = time.time()

for i in range(start_year,end_year+1):
    
    start = time.time()
    print('Processing ' + str(i)+' Data...')
    
    if i >= 2018:
        url = 'https://www.eia.gov/electricity/data/eia923/xls/f923_' + str(i) +'.zip'
        name_pos = 0
        start_line = 5
        
    elif i < 2018 and i > 2010:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f923_' + str(i) +'.zip'
        name_pos = 1
        start_line = 5

    elif i < 2011 and i > 2007:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f923_' + str(i) +'.zip'
        name_pos = 1
        start_line = 7

    elif i <= 2007 and i > 2003:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f906920_' + str(i) +'.zip'
        name_pos = 1
        start_line = 7
        
    elif i >= 2001 and i <= 2003:
        url = 'https://www.eia.gov/electricity/data/eia923/archive/xls/f906920_' + str(i) +'.zip'
        name_pos = 0
        start_line = 7
    elif i < 2001:
        url= 'https://www.eia.gov/electricity/data/eia923/archive/xls/utility/f759'+str(i)+'u.xls'
        name_pos=0
        start_line=1
        
    if i >= 2001:    
        zipped_file = ZipFile(BytesIO(urlopen(url).read()))
        reader.append(pd.read_excel(zipped_file.open(zipped_file.namelist()[name_pos]), \
                                sheet_name = 0, skiprows=start_line))
    else:
        zipped_file=urlopen(url)
        reader.append(pd.read_excel(zipped_file))
    

    # Clean up columns
    reader[i-start_year].columns = reader[i-start_year].columns.str.lower().str.replace(r'\n', ' ').str.replace('(', '').str.replace(')', '')
    
    # Update year to be 4 digit
    if i <= 2000:
        reader[i-start_year]["year"] = i
        
    # Combine monthly generation to total for 1970 - 1995
    # 1970-1990 is in kWh, 1991-2000 is in MWh
    
    if i < 1996 and i > 1989: #MWh
        reader[i-start_year]['netgen']= reader[i-start_year]['gen01']+ \
                                        reader[i-start_year]['gen02']+ \
                                        reader[i-start_year]['gen03']+ \
                                        reader[i-start_year]['gen04']+ \
                                        reader[i-start_year]['gen05']+ \
                                        reader[i-start_year]['gen06']+ \
                                        reader[i-start_year]['gen07']+ \
                                        reader[i-start_year]['gen08']+ \
                                        reader[i-start_year]['gen09']+ \
                                        reader[i-start_year]['gen10']+ \
                                        reader[i-start_year]['gen11']+ \
                                        reader[i-start_year]['gen12']
    elif i < 1990: #kWh
        reader[i-start_year]['netgen']= (reader[i-start_year]['gen01']+ \
                                        reader[i-start_year]['gen02']+ \
                                        reader[i-start_year]['gen03']+ \
                                        reader[i-start_year]['gen04']+ \
                                        reader[i-start_year]['gen05']+ \
                                        reader[i-start_year]['gen06']+ \
                                        reader[i-start_year]['gen07']+ \
                                        reader[i-start_year]['gen08']+ \
                                        reader[i-start_year]['gen09']+ \
                                        reader[i-start_year]['gen10']+ \
                                        reader[i-start_year]['gen11']+ \
                                        reader[i-start_year]['gen12']) / 1000
    
    # Get the column names to match for different version of the 923 EIA data
    if i >= 2012:
        reader[i-start_year].rename(columns={'reported fuel type code':'fueltype','net generation megawatthours': 'netgen', 'plant state': 'state'}, inplace=True)
    #elif i == 2012 or i == 2013 or i == 2014:
     #   reader[i-start_year].rename(columns={'aer fuel type code':'fueltype','net generation megawatthours': 'netgen', 'plant state': 'state'}, inplace=True)
    elif i < 2012 and i >= 2001:
        reader[i-start_year].rename(columns={'reported fuel type code':'fueltype','net generation megawatthours': 'netgen'}, inplace=True)
    elif i < 2001 and i >= 1996:
        reader[i-start_year].rename(columns={
            'fueldesc':'fueltype','netgenerat': 'netgen', 'pcode':'plant id', 'pltname':'plant name','fipst':'state'}, inplace=True)
    elif i < 1996:
        reader[i-start_year].rename(columns={'fueldesc':'fueltype', 'pcode':'plant id', 'pltname':'plant name','fipst':'state'}, inplace=True)

    # Keep only the needed headers
    reader[i-start_year]=reader[i-start_year][['plant id','plant name','state', 'netgen','fueltype','year']]
    
    end = time.time()
    print('Processing took ' + str(end - start) + ' seconds')
    
total_end = time.time()
print('All Data Processed Successfully in ' + str(total_end - total_start)  + ' seconds')


Processing 1970 Data...
Processing took 6.634488105773926 seconds
Processing 1971 Data...
Processing took 5.028601169586182 seconds
Processing 1972 Data...
Processing took 5.5257627964019775 seconds
Processing 1973 Data...
Processing took 5.438066005706787 seconds
Processing 1974 Data...
Processing took 6.393418550491333 seconds
Processing 1975 Data...
Processing took 6.170813083648682 seconds
Processing 1976 Data...
Processing took 5.340097427368164 seconds
Processing 1977 Data...
Processing took 4.966947317123413 seconds
Processing 1978 Data...
Processing took 5.288481712341309 seconds
Processing 1979 Data...
Processing took 3.892218828201294 seconds
Processing 1980 Data...
Processing took 4.614046812057495 seconds
Processing 1981 Data...
Processing took 3.8458824157714844 seconds
Processing 1982 Data...
Processing took 5.108171224594116 seconds
Processing 1983 Data...
Processing took 4.973475694656372 seconds
Processing 1984 Data...
Processing took 4.458953142166138 seconds
Processi

In [9]:
# Create Fuelcode DFs
pre_2001_fuelcodes = pd.DataFrame(data={'FuelCode':
                       ['WAT','UR','FO2','FO6','ANT','PC','BIT','LIG','OIL','NG','GEO','WOD','WAS','WI',
                        'SP','SO'],
                       'FuelDescription':
                       ['Water','Nuclear','Light Oil','Heavy Oil','Anthracite','Coke','Bituminous','Lignite',
                        'Fuel Oil','Natural Gas','Geothermal','Wood','Waste','Wind','Solar - Photovoltaic',
                        'Solar - Thermal'],
                       'PowerType':
                       ['Hydroelectric','Nuclear','Petroleum','Petroleum','Coal','Coal','Coal','Coal',
                        'Petroleum','Natural Gas','Geothermal','Biomass','Biomass','Wind','Solar',
                        'Solar'],                                        
                       'Class':
                       ['Clean','Clean','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil',
                       'Clean','Fossil','Fossil','Clean','Clean','Clean']
                      })

post_2000_fuelcodes = pd.DataFrame(data={'FuelCode':
                       ['ANT','BIT','LIG','SUB','SGC','WC','DFO','JF','KER','PC','PG','RFO','SG','WO',
                        'BFG','NG','OG','NUC','WAT','WDS','WDL','BLQ','AB','MSW','OBG','OBL','OBS','LFG',
                        'SLW','SUN','WND','GEO','PUR','WH','TDF','MWH','OTH'],
                       'FuelDescription':
                       ['Anthracite Coal','Bituminous Coal','Lignite Coal','Subbituminous Coal',
                        'Coal-Derived Synthesis Gas','Waste/Other Coal','Distillate Fuel Oil','Jet Fuel',
                        'Kerosene','Petroleum Coke','Gaseous Propane','Residual Fuel Oil','Synthesis Gas',
                        'Waste/Other Oil','Blast Furnace Gas','Natural Gas','Other Gas','Nuclear',
                        'Hydroelectric','Wood Solids','Wood Waste Liquids','Black Liquor','AB','MSW','OBG','OBL','OBS','LFG',
                        'SLW','SUN','WND','GEO','PUR','WH','TDF','MWH','OTH'],
                       'PowerType':
                       ['Coal','Coal','Coal','Coal',
                        'Coal','Coal','Petroleum','Petroleum',
                        'Petroleum','Petroleum','Petroleum','Petroleum','Petroleum',
                        'Petroleum','Natural Gas','Natural Gas','Natural Gas','Nuclear',
                        'Hydroelectric','Biomass','Biomass','Biomass','Biomass','Biomass','Biomass','Biomass','Biomass','Biomass',
                        'Biomass','Solar','Wind','Geothermal','Other','Other','Other','Other','Other'],                                         
                       'Class':
                       ['Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil',
                        'Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Clean',
                        'Clean','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil','Fossil',
                        'Fossil','Clean','Clean','Clean','Clean','Clean','Fossil','Clean','Clean'],
                      })

In [21]:
# Looking at the data, we see pre 2001 uses the State ID and post uses State Abbreviation. 
# We can merge this in using the state info DF
#
# Additionally, We can merge 

# Separate into pre and pots 2001 cohorts.
pre_2001_data = pd.concat(reader[:2001-1970], ignore_index = True)
post_2000_data = pd.concat(reader[2001-1970:], ignore_index = True)

pre_2001_data['state'] = pre_2001_data['state'].astype('int64')
stateDF['StateID'] = stateDF['StateID'].astype('int64')

pre_2001_data_state = pre_2001_data.merge(stateDF, how = 'left',left_on = 'state', right_on = 'StateID')
pre_2001_data_state['state'] = pre_2001_data_state['State']

pre_2001_data_state = pre_2001_data_state.merge(pre_2001_fuelcodes, 
                                                how = 'left',left_on = 'fueltype', right_on = 'FuelCode')
pre_2001_data_state = pre_2001_data_state[['plant id','plant name','state','netgen','fueltype',
                                           'FuelDescription','PowerType','year','Class']]

post_2000_data = post_2000_data.merge(post_2000_fuelcodes, 
                                      how = 'left',left_on = 'fueltype', right_on = 'FuelCode')

post_2000_data = post_2000_data[['plant id','plant name','state','netgen',
                                 'FuelDescription','PowerType','year','Class']]

# Concatenate each year into a master list
data = pd.concat([pre_2001_data_state,post_2000_data], ignore_index = True, sort=False)
print('Individual Dataframes Concatenated.')

# This removes the non-power plant data at the end and the "0" Plant ID data
data = data[~data['plant id'].isin([99999])]
data = data[~data['plant id'].isin([0])]

# Clean NAs and set plant ID type.
data.dropna(subset = ['plant id','plant name'], inplace = True)
data['plant id'] = data['plant id'].astype('int64')

# Sort by Plant ID, Year
data.sort_values(['plant id','year'], ascending=[True,True], inplace=True)
data.reset_index(inplace=True)

print(data.shape)

## Looking at a particular plant, you notice that the pre 2001 data has a different
## name than the post 2001 data.
##
## Also, we can see that some of the states are using the state code whereas some are 

print(data[(data['plant id'] == 2) & (data['year'] == 1970)])
print('\n')
print(data[(data['plant id'] == 2) & (data['year'] == 2001)])

Individual Dataframes Concatenated.
(350128, 10)
   index  plant id   plant name state    netgen fueltype FuelDescription  \
0   4235         2  BANKHEAD DM    AL  147701.7      WAT           Water   

       PowerType  year  Class  
0  Hydroelectric  1970  Clean  


     index  plant id    plant name state    netgen fueltype FuelDescription  \
31  172052         2  Bankhead Dam    AL  234987.0      NaN   Hydroelectric   

        PowerType  year  Class  
31  Hydroelectric  2001  Clean  


In [22]:
# Validate by checking a specific plant
print(data[(data['plant id'] == 3) & (data['year'] > 0)].head(5))

    index  plant id plant name state     netgen fueltype FuelDescription  \
48   4248         3      BARRY    AL   271265.0       PC            Coke   
49   4249         3      BARRY    AL  4959894.3      BIT      Bituminous   
50  10362         3      BARRY    AL   342611.0       PC            Coke   
51  10363         3      BARRY    AL  5186249.0      BIT      Bituminous   
52  10364         3      BARRY    AL     1887.0      OIL        Fuel Oil   

    PowerType  year   Class  
48       Coal  1970  Fossil  
49       Coal  1970  Fossil  
50       Coal  1971  Fossil  
51       Coal  1971  Fossil  
52  Petroleum  1971  Fossil  


In [24]:
# Group by Plant ID and take the most recent name
data['plant name'] = data.groupby(['plant id'])['plant name'].transform('last')
#data = data[['plant id','plant name','state','netgen','fueltype','year','Class']]
#group by Plant ID and Year
data[data['plant id'] == 3].head(5)

,index,plant id,plant name,state,netgen,fueltype,FuelDescription,PowerType,year,Class
48,4248,3,Barry,AL,271265.0,PC,Coke,Coal,1970,Fossil
49,4249,3,Barry,AL,4959894.3,BIT,Bituminous,Coal,1970,Fossil
50,10362,3,Barry,AL,342611.0,PC,Coke,Coal,1971,Fossil
51,10363,3,Barry,AL,5186249.0,BIT,Bituminous,Coal,1971,Fossil
52,10364,3,Barry,AL,1887.0,OIL,Fuel Oil,Petroleum,1971,Fossil


In [48]:
### 4/11 Update: Get a DF that finds the power

# First we group by plant ID and Power Type to get the netgen by Fuel Type (grouped into "Power Type" categories)
# over the life of the plant

source_df = data.copy()
source_data_grouped = source_df.groupby(['plant id','PowerType']).agg({'netgen': np.sum}).reset_index()
display(source_data_grouped.head(10))

# Now, we sort by plant id and net gen and group again and take the first row, which gives us the primary netgen type
source_data_grouped.sort_values(['plant id','netgen'], ascending=[True,False], inplace=True)
primary_power_type_data = source_data_grouped.groupby(['plant id']).first().reset_index()
primary_power_type_data.rename(columns={'PowerType': 'PrimaryFuelType','netgen':'TotalNetGen'}, inplace=True)  
# *JLB notes*: Right now I'm getting rid of TotalNetGen, but we may want to use this aggegate in the future
primary_power_type_data = primary_power_type_data[['plant id','PrimaryFuelType']]
display(primary_power_type_data.head(10))

,plant id,PowerType,netgen
0,2,Hydroelectric,7.864325e+06
1,3,Coal,3.884266e+08
2,3,Natural Gas,1.135368e+08
3,3,Petroleum,2.463043e+05
4,4,Hydroelectric,2.923636e+07
5,5,Coal,1.852040e+06
6,5,Natural Gas,2.125226e+06
7,5,Petroleum,2.319683e+05
8,6,Petroleum,2.388034e+05
9,7,Biomass,1.025485e+04


,plant id,PrimaryFuelType
0,2,Hydroelectric
1,3,Coal
2,4,Hydroelectric
3,5,Natural Gas
4,6,Petroleum
5,7,Coal
6,8,Coal
7,9,Natural Gas
8,10,Coal
9,11,Hydroelectric


In [7]:
# Get the EIA 860m Plant Data. This Contains Latitude & Longitude Data

url = 'https://www.eia.gov/electricity/data/eia860m/archive/xls/december_generator2018.xlsx'
response = urlopen(url)
plant_data = pd.read_excel(response, sheet_name = ['Operating','Planned','Retired'], skiprows = 1, quotechar = '"')

# Clean EIA plant data 

plant_data['Operating'] = plant_data['Operating'][
    ['Plant ID','Plant Name','Nameplate Capacity (MW)','Operating Year','County','Latitude','Longitude']]
plant_data['Planned'] = plant_data['Planned'][
    ['Plant ID','Plant Name','Nameplate Capacity (MW)','Planned Operation Year','County','Latitude','Longitude']]
plant_data['Retired'] = plant_data['Retired'][
    ['Plant ID','Plant Name','Nameplate Capacity (MW)','Operating Year','County','Latitude','Longitude']]

# Rename Columns

plant_data['Operating'].rename(
    columns={'Operating Year': 'In Service Year', 'Nameplate Capacity (MW)':'Capacity'}, inplace=True)    
plant_data['Planned'].rename(
    columns={'Planned Operation Year': 'In Service Year', 'Nameplate Capacity (MW)':'Capacity'}, inplace=True)    
plant_data['Retired'].rename(
    columns={'Operating Year': 'In Service Year', 'Nameplate Capacity (MW)':'Capacity'}, inplace=True)    

# Add Status Column
## 1 = Operating
## 0 = Planned or Retired

plant_data['Operating']['Status'] = 1
plant_data['Planned']['Status'] = 0
plant_data['Retired']['Status'] = 0

# Concatenate
all_plant_data = pd.concat(plant_data, ignore_index = True)

# Drop NA Plant IDs then set to integer

all_plant_data.dropna(subset=['Plant ID'], inplace=True)
all_plant_data['Plant ID'] = all_plant_data['Plant ID'].astype('int64')
all_plant_data['Capacity'] = all_plant_data['Capacity'].astype('float64')

all_plant_data['In Service Year'] = all_plant_data['In Service Year'].fillna('-1')
all_plant_data['In Service Year'] = all_plant_data['In Service Year'].replace(' ','-1')
all_plant_data['In Service Year'] = all_plant_data['In Service Year'].astype('int64')
all_plant_data['In Service Year'] = all_plant_data['In Service Year'].replace(-1,np.nan)

# Add dummy "Count" Column - used for grouping in next step

all_plant_data['Generator Count'] = 1

#Show results
all_plant_data.head(5)

,Plant ID,Plant Name,Capacity,In Service Year,County,Latitude,Longitude,Status,Generator Count
0,2,Bankhead Dam,53.9,1963.0,Tuscaloosa,33.4587,-87.3568,1,1
1,3,Barry,153.1,1954.0,Mobile,31.0069,-88.0103,1,1
2,3,Barry,153.1,1954.0,Mobile,31.0069,-88.0103,1,1
3,3,Barry,403.7,1969.0,Mobile,31.0069,-88.0103,1,1
4,3,Barry,788.8,1971.0,Mobile,31.0069,-88.0103,1,1


In [25]:
# Each row is a Generator - Group by Plant attributes to get Plant totals

all_plant_data_grouped = all_plant_data.groupby(
    ['Plant ID','Plant Name','County','Latitude','Longitude']).agg(
    {'Capacity': np.mean, 'In Service Year': np.min})


# Reset Hierarchal Index and clean data types 
# lowercase "plant id" this time for more of a natural join)
all_plant_data_grouped.reset_index(inplace=True)

all_plant_data_grouped['In Service Year'] = all_plant_data_grouped['In Service Year'].fillna('-1')
all_plant_data_grouped['In Service Year'] = all_plant_data_grouped['In Service Year'].replace(' ','-1')
all_plant_data_grouped['In Service Year'] = all_plant_data_grouped['In Service Year'].astype('int64')
all_plant_data_grouped = all_plant_data_grouped[all_plant_data_grouped['In Service Year'] != -1]

#all_plant_data_grouped['Latitude'] = all_plant_data_grouped['Latitude'].astype('float64')
#all_plant_data_grouped['Longitude'] = all_plant_data_grouped['Longitude'].astype('float64')
all_plant_data_grouped['Plant Name'] = all_plant_data_grouped['Plant Name'].astype(str)
all_plant_data_grouped['County'] = all_plant_data_grouped['County'].astype(str)

all_plant_data_grouped.head(5)

,Plant ID,Plant Name,County,Latitude,Longitude,Capacity,In Service Year
0,2,Bankhead Dam,Tuscaloosa,33.4587,-87.3568,53.900000,1963
1,3,Barry,Mobile,31.0069,-88.0103,258.318182,1954
2,4,Walter Bouldin Dam,Elmore,32.5839,-86.2831,75.000000,1967
3,7,Gadsden,Etowah,34.0128,-85.9708,69.000000,1949
4,8,Gorgas,Walker,33.6443,-87.1965,283.340000,1951


In [54]:
# Join Net Gen Data with Grouped Plant Coordinate Data for a master data set
# Level of detail: Row = Plant x Generator x year
combined_data = data.merge(all_plant_data_grouped, how = 'left',left_on = 'plant id', right_on = 'Plant ID')

combined_data = combined_data[
    ['plant id','plant name','state','netgen','year',
     'County','Latitude','Longitude','In Service Year','Class']]

# 4/11 Edits: Merge in new primary_power_type_data so that each plant

combined_data = combined_data.merge(primary_power_type_data, how = 'left', left_on = 'plant id', right_on = 'plant id')

# Currently we have some NaN values (first example: Plant ID 5)
# Set vals to -9999 so we can set type to int (float required if NaN values present)

combined_data['In Service Year'] = combined_data['In Service Year'].fillna('-9999')
combined_data['In Service Year'] = combined_data['In Service Year'].astype('int64')

combined_data.head(10)

,plant id,plant name,state,netgen,year,County,Latitude,Longitude,In Service Year,Class,PrimaryFuelType
0,2,Bankhead Dam,AL,147701.7,1970,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
1,2,Bankhead Dam,AL,188612.0,1971,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
2,2,Bankhead Dam,AL,158977.6,1972,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
3,2,Bankhead Dam,AL,225054.0,1973,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
4,2,Bankhead Dam,AL,195276.0,1974,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
5,2,Bankhead Dam,AL,237326.0,1975,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
6,2,Bankhead Dam,AL,171253.0,1976,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
7,2,Bankhead Dam,AL,211218.0,1977,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
8,2,Bankhead Dam,AL,129975.0,1978,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric
9,2,Bankhead Dam,AL,239343.0,1979,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric


In [55]:
# Group Combined data to level of Clean/Fossil
# Should be at most 2 x # of Plants (will likely be much closer to just total # of plants) per year

combined_data_grouped = combined_data.groupby(
    ['plant id','plant name','state','year','County','Latitude','Longitude',
     'In Service Year','Class','PrimaryFuelType']).agg({'netgen': np.sum})
combined_data_grouped.reset_index(inplace=True)
combined_data_grouped.head(5)

,plant id,plant name,state,year,County,Latitude,Longitude,In Service Year,Class,PrimaryFuelType,netgen
0,2,Bankhead Dam,AL,1970,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric,147701.7
1,2,Bankhead Dam,AL,1971,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric,188612.0
2,2,Bankhead Dam,AL,1972,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric,158977.6
3,2,Bankhead Dam,AL,1973,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric,225054.0
4,2,Bankhead Dam,AL,1974,Tuscaloosa,33.4587,-87.3568,1963,Clean,Hydroelectric,195276.0


In [56]:
# Save final Combined Grouped Datafile to CSV
combined_data_grouped.to_csv(r'powerplant_data.csv', index=None, header=True)